In [181]:
import csv
import pandas as pd
import numpy as np
import string
import sklearn
from sklearn.neural_network import MLPClassifier
from patsy import dmatrices
from sklearn import metrics
import random
import logging
from tqdm import tqdm
from functools import reduce
from operator import add

# Import and explore data

In [2]:
data = pd.read_csv('neuralnetinput.csv', sep = ',')

In [3]:
data.head(5)

,all.title,countComments,countViews,countLikes,countDislikes,Production Budget,Domestic Gross,International Gross,WorldWide Gross,Domestic Blu-Ray Sales,...,studio_Universal,studio_Walt Disney,studio_Warner Bros,studio_Weinstein Co,studio_Weinstein Co st,LDLRatio,Bing.Liu.Score,Harvard.Score,compound,ROI_RATING
0,10 Cloverfield Lane,9.005773,16.110399,10.385852,7.100852,15.424949,18.093329,17.404664,18.500290,15.217882,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.086528,1.019531,0.988208,0.907715,Great
1,12 Years a Slave,9.093132,15.713606,9.633318,6.804615,16.811243,17.852791,18.638638,19.014148,15.858818,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.079877,1.007665,0.997844,0.831040,Great
2,127 Hours,9.116030,16.351912,10.244059,6.508769,16.705882,16.724335,17.550365,17.913468,14.838388,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.090834,1.037485,1.036511,0.931896,Ok
3,17 Again,5.529429,14.220295,8.494539,4.718499,17.504390,17.977001,18.137095,18.753395,14.647010,...,0.693147,0.693147,1.098612,0.693147,0.693147,1.091246,1.093800,1.077682,0.954623,Ok
4,2 Guns,4.997212,13.720216,7.360104,4.094345,17.926384,18.141132,17.856464,18.702041,15.912304,...,1.098612,0.693147,0.693147,0.693147,0.693147,1.086666,1.093539,1.080629,0.947924,Passing


In [5]:
data.groupby('ROI_RATING').mean()

,countComments,countViews,countLikes,countDislikes,Production Budget,Domestic Gross,International Gross,WorldWide Gross,Domestic Blu-Ray Sales,Domestic DVD Sales,...,studio_Tribeca Film / Well Go USA,studio_Universal,studio_Walt Disney,studio_Warner Bros,studio_Weinstein Co,studio_Weinstein Co st,LDLRatio,Bing.Liu.Score,Harvard.Score,compound
ROI_RATING,,,,,,,,,,,,,,,,,,,,,
Great,7.518705,15.448825,9.390606,6.540355,16.714206,18.062350,17.960985,18.795540,15.450580,16.429942,...,0.693147,0.753859,0.719166,0.747354,0.714830,0.695315,1.077336,1.053603,1.039585,0.917707
Ok,7.369955,15.283389,9.177029,6.313327,17.572821,17.945104,17.917203,18.740038,15.567891,16.364658,...,0.693147,0.747645,0.734566,0.749825,0.706227,0.693147,1.077809,1.048818,1.037082,0.922949
Passing,6.688418,14.679852,8.361911,5.683610,17.484086,17.274205,16.671310,18.100302,15.025111,16.021711,...,0.693147,0.734566,0.728026,0.758545,0.701867,0.693147,1.074897,1.065508,1.051253,0.925186
Poor,6.643703,14.582235,8.255354,5.777888,16.980159,14.040935,14.273176,15.436037,13.640603,14.646158,...,0.695327,0.730206,0.710587,0.745465,0.701867,0.693147,1.066245,1.065892,1.050520,0.907078


In [9]:
allcols = list(data)

In [81]:
dontinclude = ['alltitle','ROI_RATING', 'ROI', 'studio_']

In [82]:
#create formula
features = []
#rename columns
for word in allcols:
    newword = ''
    for letter in word:
        if letter == '.':
            pass
        elif letter == '/':
            pass
        elif letter == '-':
            pass
        elif letter == ' ':
            pass
        elif letter == '!':
            pass
        else:
            newword = newword + letter
    print newword
    features.append(newword)

alltitle
countComments
countViews
countLikes
countDislikes
ProductionBudget
DomesticGross
InternationalGross
WorldWideGross
DomesticBluRaySales
DomesticDVDSales
TotalDomesticVideoSales
ROI
genre_Action
genre_Adventure
genre_BlackComedy
genre_Comedy
genre_ConcertPerformance
genre_Documentary
genre_Drama
genre_Horror
genre_Musical
genre_RomanticComedy
genre_ThrillerSuspense
mpaa_G
mpaa_NC17
mpaa_Not
mpaa_PG
mpaa_PG13
studio_
studio_20thCenturyFox
studio_A
studio_Alchemy
studio_AnchorBayEntertainment
studio_Apparition
studio_BroadGreenPictures
studio_CBSFilms
studio_CinedigmFilmArcade
studio_ClariusEntertainment
studio_ElevationPicturesCanada
studio_EntertainmentOne
studio_EuropaCorp
studio_FilmDistrict
studio_FocusGramercy
studio_FocusFeatures
studio_FoxSearchlight
studio_FreestyleReleasing
studio_IFCFilms
studio_InvinciblePictures
studio_Lionsgate
studio_Lionsgatend
studio_LionsgatePremiere
studio_Lionsgatest
studio_MagnoliaPictures
studio_MGM
studio_Miramax
studio_MomentumPictures
stud

In [83]:
data.columns = features

In [85]:
finalfeatures = []
for i in features:
    if i in dontinclude:
        pass
    else:
        finalfeatures.append(i)
form = '+'.join(finalfeatures)
form = 'ROI_RATING'+'~'+form
form

'ROI_RATING~countComments+countViews+countLikes+countDislikes+ProductionBudget+DomesticGross+InternationalGross+WorldWideGross+DomesticBluRaySales+DomesticDVDSales+TotalDomesticVideoSales+genre_Action+genre_Adventure+genre_BlackComedy+genre_Comedy+genre_ConcertPerformance+genre_Documentary+genre_Drama+genre_Horror+genre_Musical+genre_RomanticComedy+genre_ThrillerSuspense+mpaa_G+mpaa_NC17+mpaa_Not+mpaa_PG+mpaa_PG13+studio_20thCenturyFox+studio_A+studio_Alchemy+studio_AnchorBayEntertainment+studio_Apparition+studio_BroadGreenPictures+studio_CBSFilms+studio_CinedigmFilmArcade+studio_ClariusEntertainment+studio_ElevationPicturesCanada+studio_EntertainmentOne+studio_EuropaCorp+studio_FilmDistrict+studio_FocusGramercy+studio_FocusFeatures+studio_FoxSearchlight+studio_FreestyleReleasing+studio_IFCFilms+studio_InvinciblePictures+studio_Lionsgate+studio_Lionsgatend+studio_LionsgatePremiere+studio_Lionsgatest+studio_MagnoliaPictures+studio_MGM+studio_Miramax+studio_MomentumPictures+studio_OpenRo

In [92]:
data['ROI_RATING'].unique()

array(['Great', 'Ok', 'Passing', 'Poor'], dtype=object)

In [103]:
#encode targetvariables into labels (0,1,2,3)
le = sklearn.preprocessing.LabelEncoder()
le.fit(data['ROI_RATING'].unique())
list(le.classes_)
data['ROI_RATING'] = le.transform(data['ROI_RATING'])

In [104]:
data.head(5)

,alltitle,countComments,countViews,countLikes,countDislikes,ProductionBudget,DomesticGross,InternationalGross,WorldWideGross,DomesticBluRaySales,...,studio_Universal,studio_WaltDisney,studio_WarnerBros,studio_WeinsteinCo,studio_WeinsteinCost,LDLRatio,BingLiuScore,HarvardScore,compound,ROI_RATING
0,10 Cloverfield Lane,9.005773,16.110399,10.385852,7.100852,15.424949,18.093329,17.404664,18.500290,15.217882,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.086528,1.019531,0.988208,0.907715,0
1,12 Years a Slave,9.093132,15.713606,9.633318,6.804615,16.811243,17.852791,18.638638,19.014148,15.858818,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.079877,1.007665,0.997844,0.831040,0
2,127 Hours,9.116030,16.351912,10.244059,6.508769,16.705882,16.724335,17.550365,17.913468,14.838388,...,0.693147,0.693147,0.693147,0.693147,0.693147,1.090834,1.037485,1.036511,0.931896,1
3,17 Again,5.529429,14.220295,8.494539,4.718499,17.504390,17.977001,18.137095,18.753395,14.647010,...,0.693147,0.693147,1.098612,0.693147,0.693147,1.091246,1.093800,1.077682,0.954623,1
4,2 Guns,4.997212,13.720216,7.360104,4.094345,17.926384,18.141132,17.856464,18.702041,15.912304,...,1.098612,0.693147,0.693147,0.693147,0.693147,1.086666,1.093539,1.080629,0.947924,2


# SPLIT DATA

In [110]:
#split data
y,x = dmatrices(form, data, return_type = 'dataframe')
y = np.ravel(y)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size = 0.3, random_state = 0)

# FIRST RUN

In [128]:
clf = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = 100, max_iter = 100, random_state = 1, verbose = True)

In [129]:
clf.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=100, learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=True,
       warm_start=False)

In [130]:
nn_predicted = clf.predict(x_test)

In [133]:
print metrics.accuracy_score(y_test,nn_predicted)
#print metrics.roc_auc_score(y_test, probability[:,1])
print(metrics.confusion_matrix(y_test,nn_predicted))
print(metrics.classification_report(y_test,nn_predicted))
pd.crosstab(y_test,nn_predicted, rownames = ['Actual'],colnames = ['Predicted'], margins = True)

0.883928571429
[[53  6  0  0]
 [ 3 43  4  0]
 [ 0  5 42  3]
 [ 0  0  5 60]]
             precision    recall  f1-score   support

        0.0       0.95      0.90      0.92        59
        1.0       0.80      0.86      0.83        50
        2.0       0.82      0.84      0.83        50
        3.0       0.95      0.92      0.94        65

avg / total       0.89      0.88      0.89       224



Predicted,0.0,1.0,2.0,3.0,All
Actual,,,,,
0.0,53,6,0,0,59
1.0,3,43,4,0,50
2.0,0,5,42,3,50
3.0,0,0,5,60,65
All,56,54,51,63,224


In [172]:
#get prec, recall, fscore, support for each NN
print metrics.classification_report(y_test,nn_predicted)
zzz = metrics.precision_recall_fscore_support(y_test, nn_predicted)
for i in range(len(zzz)):
    if i == 3:
        print zzz[i], zzz[i].sum()
    else:
        print zzz[i], zzz[i].mean()

             precision    recall  f1-score   support

        0.0       0.95      0.90      0.92        59
        1.0       0.80      0.86      0.83        50
        2.0       0.82      0.84      0.83        50
        3.0       0.95      0.92      0.94        65

avg / total       0.89      0.88      0.89       224

[ 0.94642857  0.7962963   0.82352941  0.95238095] 0.879658807968
[ 0.89830508  0.86        0.84        0.92307692] 0.880345501956
[ 0.92173913  0.82692308  0.83168317  0.9375    ] 0.879461343919
[59 50 50 65] 224


# GENETIC ALGO IMPLEMENTATION

In [278]:
'''
    Inspired by: https://github.com/harvitronix/neural-network-genetic-algorithm
    MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=100, learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=True,
       warm_start=False)
'''

class NN():
    def __init__(self, nn_params, x_train, x_test, y_train, y_test):
        self.accuracy = 0
        self.params = nn_params
        self.network = {}
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
    
    def createNewRandom(self, ):
        for key in self.params:
            self.network[key] = random.choice(self.params.get(key))
        #create nnet
        
        
    
    def importSettings(self, settings):
            self.network = settings
    
    def train(self):
        if self.accuracy == 0:
            #fit training data to net
            clf = MLPClassifier(solver = self.network.get('solver'), 
                            hidden_layer_sizes = self.network.get('hidden_layer_sizes'), 
                            max_iter = self.network.get('max_iter'), 
                            #activation = self.network.get('activation'),
                            learning_rate_init = self.network.get('learning_rate_init'))
            clf.fit(self.x_train, self.y_train)
            #print 'Network fitted...'
            nn_predicted = clf.predict(self.x_test)
            #print 'Predictions created...'
            
            self.accuracy = metrics.accuracy_score(y_test,nn_predicted)
            #print 'Accuracy stored.'
            
            other_metrics = metrics.precision_recall_fscore_support(y_test, nn_predicted)
            #print 'Calculating other metrics...'
            for i in range(len(other_metrics)):
                if i == 0:
                    self.precision = (other_metrics[i].astype(np.float)).mean()
                    #print 'Precision per Label:', other_metrics[i], '| Ave. Precision:', other_metrics[i].mean()
                elif i == 1:
                    self.recall = (other_metrics[i].astype(np.float)).mean()
                    #print 'Recall per Label:', other_metrics[i], '| Ave. Recall:', other_metrics[i].mean()
                elif i == 2:
                    self.f1score = (other_metrics[i].astype(np.float)).mean()
                    #print 'F1 Score per Label:', other_metrics[i], '| Ave. F1 Score:', other_metrics[i].mean()
                else:
                    self.support = (other_metrics[i].astype(np.float)).sum()
                    #print 'Support per Label:', other_metrics[i], '| Total Support:', other_metrics[i].mean()
            #print 'Network accuracy:', self.accuracy
            #print(metrics.classification_report(y_test,nn_predicted))
            pd.crosstab(y_test,nn_predicted, rownames = ['Actual'],colnames = ['Predicted'], margins = True)
        
    def print_network(self):
        logging.info(self.network)
        logging.info('Network Accuracy: %.2f%%' % (self.accuracy * 100))
        logging.info('Precision: %d | Recall: %d | F1 Score: %d | Support: %d' % (self.precision, self.recall, self.f1score, self.support))
        
        
class Optimizer():
    #def __init__(self, nn_params, retain = 0.4, random_select = 0.1, mutate_chance = 0.2, x_train, x_test, y_train, y_test):
    def __init__(self, nn_params, retain, random_select, mutate_chance, x_train, x_test, y_train, y_test):
        self.mutate_chance = mutate_chance
        self.retain = retain
        self.random_select = random_select
        self.nn_params = nn_params
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test
        
    def create_pop(self, n):
        pop = []
        for i in range(n):
            #print 'Networks created:', i + 1
            network = NN(self.nn_params, self.x_train, self.y_train, self.x_test, self.y_test)
            network.createNewRandom()
            pop.append(network)
        return pop
    
    @staticmethod
    def fitness(network):
        return network.accuracy
    
    def grade(self, pop):
        total = reduce(add, (self.fitness(network) for network in pop))
        return total/float(len(pop))
    
    def breed(self, mother, father):
        '''
        Args: mother(dict) = network params
              father(dict) = network params  
        '''
        children = []
        for i in range(2):
            child = {}
            
            for param in self.nn_params:
                child[param] = random.choice(
                                            [mother.network[param],father.network[param]]
                                            )
            network = NN(self.nn_params, self.x_train, self.y_train, self.x_test, self.y_test)
            network.importSettings(child)
            
            children.append(network)
        return children
            
    def mutate(self, network):
        mutation = random.choice(list(self.nn_params.keys()))
        network.network[mutation] = random.choice(self.nn_params[mutation])
        
        return network
    
    def evolve(self, pop):
        scores = [(self.fitness(network), network) for network in pop]
        scores = [x[1] for x in sorted(scores, key = lambda x: x[0], reverse = True)]
        #print 'Scores:', scores
        
        retain_length = int(len(scores)*self.retain)
        
        parents = scores[:retain_length]
        
        
        for unfit in scores[retain_length:]:
            if self.random_select > random.random():
                parents.append(unfit)
        
        for parent in parents:
            if self.mutate_chance > random.random():
                parent = self.mutate(parent)
#         print 'Mutation done...'
#         print 'Parents:', parents
        parents_length = len(parents)
        desired_length = len(pop) - parents_length
        children = []
#         print 'Desired length:', desired_length
        
        while len(children) < desired_length:
            male = random.randint(0, parents_length - 1)
            female = random.randint(0, parents_length - 1)
            
            if male != female:
                male = parents[male]
                female = parents[female]
                
                babies = self.breed(male, female)
                
                for baby in babies:
                    if len(children) < desired_length:
                        children.append(baby)
        
        parents.extend(children)
        
        return parents
            
    
    
def generate(generations, population, nn_params, retain, random_select, mutate_chance, x_train, y_train, x_test, y_test):
    optimizer = Optimizer(nn_params, retain, random_select, mutate_chance, x_train, y_train, x_test, y_test)
    #print 'Optimizer created.'
    networks = optimizer.create_pop(population)
    #print 'Initial network population created.'
    for generation in range(generations):
        logging.info('Generation: %d of %d' % (generation+1, generations))
        
        #train networks in population
        progressbar = tqdm(total = len(networks))
        for network in networks:
            network.train()
            progressbar.update(1)
            progressbar.set_description('Generation: %d of %d' % (generation+1, generations))
            progressbar.refresh()
        progressbar.close()
        
        #compute for accuracy
        total_acc = 0
        for network in networks:
            total_acc += network.accuracy
        ave_acc = total_acc / len(networks)
        
        logging.info('Generation average: %.2f%%' % (ave_acc * 100))
        logging.info('-----------------------------------------------------------')
        
        if generation != generations - 1:
            networks = optimizer.evolve(networks)
    
    networks = sorted(networks, key = lambda x: x.accuracy, reverse = True)
    
    logging.info('-----------------------------------------------------------')
    for network in networks[:5]:
        network.print_network()
        

def format_params(nn_params):
    hidden_layers = nn_params.get('hidden_layers')
    num_of_neurons_per_hidden_layer = nn_params.get('num_of_neurons_per_hidden_layer')
    
    params = []
    
    for i in hidden_layers:
        for j in num_of_neurons_per_hidden_layer:
            if i == 1:
                hidden_layer_sizes = tuple((j,))
            else:
                hidden_layer_sizes = tuple((j,j))
            params.append(hidden_layer_sizes)
    nn_params['hidden_layer_sizes'] = params
    nn_params.pop('hidden_layers', None)
    nn_params.pop('num_of_neurons_per_hidden_layer', None)
    return nn_params



In [279]:
# Setup logging.
reload(logging)
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.DEBUG,
    filename='log.txt'
)

generations = 10
population = 20
nn_params = {'solver':['lbfgs', 'sgd', 'adam'],
             'hidden_layers':[1,2],
             'num_of_neurons_per_hidden_layer':[10,20,30,40,50,60,70,80,90,100,250,500,750,1000],
             'max_iter':[20,50,100,150,200,500],
             #'activation':['identity', 'logistic', 'tanh', 'relu'],
             'learning_rate_init':[0.001, 0.0001, 0.01, 0.1],
             'hidden_layer_sizes':''}

#format hidden_layer_sizes for sklearn.neural_network.MLPClassifier
nn_params = format_params(nn_params)
#print 'Parameters formatted.'

logging.info('Evolving %d generations starting with %d neural networks' % (generations, population))

generate(generations, population, nn_params, 0.25, 0.1, 0.2, x_train, y_train, x_test, y_test)
    




  0%|          | 0/20 [00:00<?, ?it/s]


  5%|▌         | 1/20 [00:00<00:06,  2.93it/s]


Generation: 1 of 10:   5%|▌         | 1/20 [00:00<00:06,  2.93it/s]


Generation: 1 of 10:  10%|█         | 2/20 [00:00<00:06,  2.93it/s]


Generation: 1 of 10:  15%|█▌        | 3/20 [00:00<00:05,  3.06it/s]


Generation: 1 of 10:  15%|█▌        | 3/20 [00:00<00:05,  3.06it/s]


Generation: 1 of 10:  20%|██        | 4/20 [00:01<00:06,  2.36it/s]


Generation: 1 of 10:  20%|██        | 4/20 [00:01<00:06,  2.36it/s]


Generation: 1 of 10:  25%|██▌       | 5/20 [00:01<00:05,  2.77it/s]


Generation: 1 of 10:  25%|██▌       | 5/20 [00:01<00:05,  2.77it/s]


Generation: 1 of 10:  30%|███       | 6/20 [00:02<00:05,  2.51it/s]


Generation: 1 of 10:  30%|███       | 6/20 [00:02<00:05,  2.51it/s]


Generation: 1 of 10:  35%|███▌      | 7/20 [00:02<00:05,  2.29it/s]


Generation: 1 of 10:  35%|███▌      | 7/20 [00:02<00:05,  2.29it/s]


Generation: 1 of 10:  40%|████      | 8/20 [00:02<00:05,  2.29it/s]

# Results
### First Run
Accuracy: 0.883928571429 ~ 88.40 %

### Genetic Algorithm Method
06/19/2017 09:34:47 PM - INFO - Evolving 10 generations starting with 20 neural networks

06/19/2017 09:34:47 PM - INFO - Generation: 1 of 10

06/19/2017 09:35:06 PM - INFO - Generation average: 50.36%

06/19/2017 09:35:06 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:06 PM - INFO - Generation: 2 of 10

06/19/2017 09:35:13 PM - INFO - Generation average: 83.44%

06/19/2017 09:35:13 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:13 PM - INFO - Generation: 3 of 10

06/19/2017 09:35:22 PM - INFO - Generation average: 82.72%

06/19/2017 09:35:22 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:22 PM - INFO - Generation: 4 of 10

06/19/2017 09:35:34 PM - INFO - Generation average: 94.20%

06/19/2017 09:35:34 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:34 PM - INFO - Generation: 5 of 10

06/19/2017 09:35:40 PM - INFO - Generation average: 88.28%

06/19/2017 09:35:40 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:40 PM - INFO - Generation: 6 of 10

06/19/2017 09:35:46 PM - INFO - Generation average: 84.93%

06/19/2017 09:35:46 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:46 PM - INFO - Generation: 7 of 10

06/19/2017 09:35:53 PM - INFO - Generation average: 91.81%

06/19/2017 09:35:53 PM - INFO - -----------------------------------------------------------

06/19/2017 09:35:53 PM - INFO - Generation: 8 of 10

06/19/2017 09:36:01 PM - INFO - Generation average: 80.47%

06/19/2017 09:36:01 PM - INFO - -----------------------------------------------------------

06/19/2017 09:36:01 PM - INFO - Generation: 9 of 10

06/19/2017 09:36:05 PM - INFO - Generation average: 79.46%

06/19/2017 09:36:05 PM - INFO - -----------------------------------------------------------

06/19/2017 09:36:05 PM - INFO - Generation: 10 of 10

06/19/2017 09:36:12 PM - INFO - Generation average: 92.28%

06/19/2017 09:36:12 PM - INFO - -----------------------------------------------------------

### TOP 5 NETWORKS
1. 06/19/2017 09:36:12 PM - INFO - {'solver': 'lbfgs', 'max_iter': 500, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': (10,)}
    - 06/19/2017 09:36:12 PM - INFO - Network Accuracy: 98.66%
    - 06/19/2017 09:36:12 PM - INFO - Precision: 0 | Recall: 0 | F1 Score: 0 | Support: 224 <- obviously this isn't working properly lol

2. 06/19/2017 09:36:12 PM - INFO - {'solver': 'lbfgs', 'max_iter': 500, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': (10,)}
    - 06/19/2017 09:36:12 PM - INFO - Network Accuracy: 98.21%
    - 06/19/2017 09:36:12 PM - INFO - Precision: 0 | Recall: 0 | F1 Score: 0 | Support: 224

3. 06/19/2017 09:36:12 PM - INFO - {'solver': 'lbfgs', 'max_iter': 500, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': (10,)}
    - 06/19/2017 09:36:12 PM - INFO - Network Accuracy: 98.21%
    - 06/19/2017 09:36:12 PM - INFO - Precision: 0 | Recall: 0 | F1 Score: 0 | Support: 224
    
4. 06/19/2017 09:36:12 PM - INFO - {'solver': 'sgd', 'max_iter': 500, 'learning_rate_init': 0.1, 'hidden_layer_sizes': (10,)}
    - 06/19/2017 09:36:12 PM - INFO - Network Accuracy: 98.21%
    - 06/19/2017 09:36:12 PM - INFO - Precision: 0 | Recall: 0 | F1 Score: 0 | Support: 224

5. 06/19/2017 09:36:12 PM - INFO - {'solver': 'lbfgs', 'max_iter': 500, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': (10,)}
    - 06/19/2017 09:36:12 PM - INFO - Network Accuracy: 98.21%
    - 06/19/2017 09:36:12 PM - INFO - Precision: 0 | Recall: 0 | F1 Score: 0 | Support: 224
